### PyEmma Featurizer Support

In [1]:
import openpathsampling as paths

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
#! lazy
storage = paths.Storage('engine_store_test.nc', mode='a')

Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [4]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [5]:
cv = paths.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [6]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [7]:
#! lazy
print storage.save(cv)

(store.cvs[CollectiveVariable], 6, 0)


and apply the featurizer to a trajectory

In [8]:
cv(storage.trajectories[1]);

Sync to make sure the cache is written to the netCDF file.

In [9]:
storage.sync_all()

In [10]:
cv(storage.snapshots.all());

In [11]:
storage.cvs.sync()

In [12]:
storage.sync_all()

In [13]:
py_cv = storage.cvs['pyemma']

In [14]:
nc_var = storage.variables['snapshots_cv_%d_values_fw' % storage.idx(py_cv)]

In [15]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [16]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [17]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_PyEMMA_Featurizer", "_dict": {"cv_store_cache": true, "topology": {"_obj": "topologies", "_idx": 0}, "name": "pyemma", "featurizer": {"_marshal": "YwEAAAABAAAAAwAAAEMAAABzIAAAAHwAAGoAAHwAAGoBAHwAAGoCAIMAAIMBAIMBAAFkAABTKAEAAABOKAMAAAB0FQAAAGFkZF9pbnZlcnNlX2Rpc3RhbmNlc3QFAAAAcGFpcnN0DwAAAHNlbGVjdF9CYWNrYm9uZSgBAAAAdAEAAABmKAAAAAAoAAAAAHMeAAAAPGlweXRob24taW5wdXQtNC00Nzc0ZDhlZGRkMDA+dBAAAABweWVtbWFfZ2VuZXJhdG9yAQAAAHMCAAAAAAE=", "_module_vars": [], "_global_vars": []}, "kwargs": {}}}']


In [18]:
py_cv_idx = storage.idx(py_cv)
print py_cv_idx
py_emma_feat = storage.vars['cvs_json'][py_cv_idx]

0


In [19]:
erg = py_emma_feat(storage.snapshots);

In [20]:
#! lazy
print erg[0::25,2:4]

[[ 2.68972969  2.06547379]
 [ 2.61396813  2.00944018]]
